# Crossmap Code

## Setup

### Install dependencies

In [ ]:
!pip install qiskit
!pip install qiskit_optimization
!pip install qiskit_aer
!pip install qiskit_ionq

In [ ]:
# Uncomment the lines below if you have access to an IONQ backend (specified in .env)

# import os
# from dotenv import load_dotenv
# load_dotenv()
# from qiskit_ionq import IonQProvider
# provider = IonQProvider(os.getenv('IONQ_PROVIDER_TOKEN'))
# backend = provider.get_backend("ionq_simulator")  # use this one to access a simulator of a quantum computer
# backend = provider.get_backend("ionq_qpu.aria-1") # use this one to access the actual quantum hardware called "aria-1"

In [10]:
from qiskit import *
from qiskit.visualization import *
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import MinimumEigenOptimizer


### Utility functions (for the algorithm)

In [11]:
import numpy as np
from qiskit_optimization.algorithms import ADMMOptimizer
from qiskit_optimization import QuadraticProgram
from qiskit_aer import Aer
from qiskit.primitives import Sampler
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import ADMMOptimizer, MinimumEigenOptimizer, ADMMParameters
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA
qp = QuadraticProgram()

In [ ]:
# Create a 2D matrix filled with 0s and place nodes with optional radius metric
# Returns an 'rows' by 'cols' 2D Array 
def create_matrix_with_nodes(rows, cols, node_placements, radius=None):
    # Initialize the matrix with 0s
    matrix = [[0 for _ in range(cols)] for _ in range(rows)]

    # Helper function to check if coordinates are within matrix bounds
    def is_within_bounds(x, y):
        return 0 <= x < rows and 0 <= y < cols

    # Place nodes in the matrix
    for x0, y0 in node_placements:
        if is_within_bounds(x0, y0):
            # If radius is specified, limit the distance expansion
            limit_radius = radius if radius is not None else max(rows, cols)
            # Determine the bounding box
            x_min = max(0, x0 - limit_radius)
            x_max = min(rows - 1, x0 + limit_radius)
            y_min = max(0, y0 - limit_radius)
            y_max = min(cols - 1, y0 + limit_radius)
            # Iterate over the bounding box
            for x in range(x_min, x_max + 1):
                for y in range(y_min, y_max + 1):
                    if matrix[x][y] == 0:
                        # Compute Euclidean distance
                        dist = ((x - x0)**2 + (y - y0)**2)**0.5
                        # Calculate the value to be assigned
                        value = max(0, int(round(limit_radius - dist)))
                        if value > 0:
                            matrix[x][y] = value
    return matrix

In [ ]:
# Create a list of 3-size tuples, where 
# each tuple contains (x, y, r) of a node 
# read from the CSV file passed into the csv file
def create_matrix_from_CSV(File file):
    # node_placements = []
    # for each CSV line in the file:
        # x = # read CSV line for x
        # y = # read CSV line for y
        # r = # read for radius

        # ret.append((x, y, r))
    return ret

In [ ]:
# Create a 1D "flattened" representation of a
# 'rows' by 'cols' Matrix with all of its entries filled

D = []
j = []
count = 0
for x in range(rows):
  for y in range(cols):
    count+=1
    matrix = create_matrix_with_nodes(rows, cols, [(x,y)], radius)
    for i in matrix:
      j = j + i
    D.append(j)
    j = []
print(D)
print(j)

[[2, 1, 0, 1, 1, 0, 0, 0, 0], [1, 2, 1, 1, 1, 1, 0, 0, 0], [0, 1, 2, 0, 1, 1, 0, 0, 0], [1, 1, 0, 2, 1, 0, 1, 1, 0], [1, 1, 1, 1, 2, 1, 1, 1, 1], [0, 1, 1, 0, 1, 2, 0, 1, 1], [0, 0, 0, 1, 1, 0, 2, 1, 0], [0, 0, 0, 1, 1, 1, 1, 2, 1], [0, 0, 0, 0, 1, 1, 0, 1, 2]]
[]


In [ ]:
# Unpack a 1D array back into CSV data
# PLACEHOLDER

### Utility functions (for printing)

In [ ]:
# PLACEHOLDER

## ADMM Algorithm
Run this to execute the Quantum optimization algorithm.

In [ ]:
D  = np.asarray(D)
rho = 1.1
mu0 = 1
k = 0
# Define the function L(x, z, lambda, mu) you want to optimize
def L(x, z, lambda_, mu):
    term1 = np.transpose(np.ones(len(D))) * x  # (1^T)x
    term2 = np.transpose(lambda_)*D*x
    term3 = (mu/2)*(np.transpose(x)*np.transpose(D)*D*x - np.transpose(np.ones(len(D)) + z)*D*x)
    return term1 + term2 + term3

# Define the constraint function c(x, z)
def c(x,z):
    x_reshaped = x.reshape((-1, 1))  # Reshape x to be compatible with D
    return np.dot(D, x_reshaped) - np.ones_like(z) - z  # Use np.ones_like to match z's sha

def Theta(z):
    return np.array([1 if zi < 0 else 0 for zi in z])

# Initialize parameters
z_star = np.zeros_like(D)
lambda_star = np.zeros_like(D)
mu_star = mu0

# Set up the quantum instance
rho_initial = 1.1
factor_c = 10.0
beta = 100.0
maxiter = 100
admm_params = ADMMParameters(rho_initial=rho_initial, factor_c=factor_c, beta=beta, maxiter=maxiter)
sampler = Sampler()
qubo_optimizer = MinimumEigenOptimizer(QAOA(sampler = sampler, optimizer=COBYLA()))

admm_optimizer = ADMMOptimizer(params=admm_params, qubo_optimizer=qubo_optimizer)


'''
rhs = 1
linear_coeffs = {}
for i in range(len(D)):
    linear_coeffs[f'x_{i}'] = D[i, 0]  # Access the first element of the array
qp.linear_constraint(linear=linear_coeffs, sense='==', rhs=rhs)
'''
# Initialize the optimizer

# Optimization loop
converged = False
while not converged:
  k += 1
  qp = QuadraticProgram()
  # Add variables to qp as necessary
  for i in range(len(D)):
      qp.binary_var(name=f'x_{i}')
  # Linear terms
  linear_terms = {}
  for i in range(len(D)):
      # Calculate the dot product and store the result as a single number
      linear_terms[f'x_{i}'] = 1 + np.dot(lambda_star[i, :], D[:, i]) - 0.5 * mu_star * np.dot((1 + z_star[i, :]), D[:, i])  # Use slicing instead of flattening

  # Quadratic terms
  quadratic_terms = {}
  for i in range(len(D)):
      for j in range(i, len(D)):
          quadratic_terms[(f'x_{i}', f'x_{j}')] = 0.5 * mu_star * np.dot(D[:, i], D[:, j])

  # Define the objective function and constraints in qp
  #qp.minimize(lambda x: L(x, z_star, lambda_star, mu_star))
  qp.minimize(linear=linear_terms, quadratic=quadratic_terms)

  # Step 5: Use a quantum annealer (QAOA here) to minimize L(x, z*, lambda*, mu*)
  result = admm_optimizer.solve(qp)
  if (k == 1):
      circuit = qubo_optimizer.min_eigen_solver.ansatz
      print(circuit)
  x_star = result.x[:9]
  print(x_star)
  # Step 6: Optimization for z
  z_temp = z_star
  z_star = np.maximum(np.zeros_like(D), c(x_star, z_star)+z_star)
  #print("z",z_star)
  # Step 7: Update lambda_star
  lambda_star = lambda_star + mu_star * c(x_star, z_star)
  #print(lambda_star)
  # Step 8: Update mu_star (this depends on your specific update rule)
  ten_percent_mu = 0.1 * mu_star
  #print(np.linalg.norm(c(x_star, z_star)))
  #print(np.linalg.norm(D * -1*(z_temp - z_star)))
  if np.linalg.norm(c(x_star, z_star)) > 10 * mu_star * np.linalg.norm(D * -1*(z_temp - z_star)):
    mu_star = rho*mu_star
  elif 10 * mu_star * np.linalg.norm(c(x_star, z_star)) < np.linalg.norm(D * -1*(z_temp - z_star)):
    mu_star = mu_star/rho

  # Step 9: Check convergence criterion
  if k == 5:
    converged = True
# Output the optimized values
print(f"x*: {x_star}) #, z*: {z_star}, lambda*: {lambda_star}, mu*: {mu_star}")

print(admm_optimizer)

## Print Results 
Plots the result from running the above algorithm.

In [ ]:
# PLACEHOLDER